In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import lu_factor, lu_solve
from scipy.linalg import qr, solve_triangular
import schemdraw
import schemdraw.elements as elm

# Here is where I import the data and format it into multiple data structures.
This first block is where I read the data and set up the coordinate dictionary

## Completed at the end of this block: 
* Data is in an numpy array of strings (data)
* The number of nodes is stored in an int (num_nodes)
* Coordinates are stored in a dictionary (coord)
    * keys: node_ids as strings 
    * values: lists of doubles
* Node IDs in a list at the same indices as in the dictionary (node_ids)
* Processed lines from data array are removed

In [ ]:
file_name = "test3.txt"
data = np.array(open("test-data/"+file_name, "r").readlines()) # Data imported into a numpy array

num_nodes = int(data[0].replace("\n","")) # The first value is the number of nodes
coord = {} # Stores all of the coordinates in a dictionary sorted by the node_id
node_ids = [] # List so index of the node_id can be accessed in the dictionary

# Removes the spaces in the array
for i in range(1, num_nodes+1):
    coord.update({data[i].split(" ")[0]: data[i].split(" ")[1:]}) 
    node_ids.append(data[i].split(" ")[0]) # The node_id is appended to the node_ids list


# Removes the "\n" and type-casts all values to ints
for i in coord:
    for j in range(len(coord.get(i))):
        coord.get(i)[j] = float(coord.get(i)[j].replace("\n",""))
print("What the coordinate dictionary looks like currently: \n" + str(coord))
data = np.delete(data, np.arange(0, num_nodes + 1)) # Removes processed data from the array




## At the end of this block:

* The number of edges is stored in an int (num_edges)
* Node-Arc Incidence Matrix stored in a numpy array (a)
    * Start node is denoted by -1, end node is denoted by 1
    * Rows represent edges, columns represent nodes
* Resistance Matrix stored in a numpy array (r)
    * Resistances on the diagonal, square matrix of edges
* Source connections stored in a dictionary (source_connect)
    * Index of row in Node-Arc Incidence Matrix as key
    * Replacement value as values
* Source Node stored as a list of the ID and the voltage (v0)
* Sink Node stored as a string of the node ID (sink)

In [ ]:
num_edges = int(data[0]) # The first remaining value is the number of edges
data = np.delete(data, 0) # Since it has been processed, it's safe to be removed
a = np.zeros((num_edges, num_nodes)) # The node-arc incidence matrix
r = np.diagflat(np.ones(num_edges)) # The resistance matrix
source_connect = {} # Keeps track of where the source connects to update the rhs later

v0 = [data[num_edges].split(" ")[0], int(data[num_edges].split(" ")[-1])] # Source node
sink = data[num_edges+1].split(" ")[0].replace("\n", "") # Sink node
for i in range(0, num_edges):
    curr = data[i].split(" ") # Creates a list that can be more easily accessed
    start = int(node_ids.index(curr[0])) # Start node_id index of the edge
    end = int(node_ids.index(curr[1])) # End node_id index of the edge
    
    a[i, start] = -1 # Updates start node in the node-arc array
    a[i, end] = 1 # Updates end node in the node-arc array
    
    
    if (curr[0] == v0[0]): # If the source node is the start of the current edge
        source_connect.update({i : v0[1]})
    if (curr[1] == v0[0]): # If the source node is the end of the current edge
        source_connect.update({i : -v0[1]})
    
    resistance = int(curr[2].replace("\n","")) # resistance of the current edge
    r[i] = r[i] * resistance # Updates the edge in the resistance array

print("Node-arc Incidence Matrix: \n"+str(a)+"\n")
print("Resistance Matrix: \n"+str(r)+"\n")
print("Source-Connection Dictionary: "+str(source_connect))



Checking the Source and Sink variables:


In [ ]:
print("Source Index:",v0[0],"at coordinates", coord.get(v0[0]), "with", v0[1], "volts")
print("Sink Index:", sink, "at coordinates", coord.get(sink))


## Schematic from data reading

In [ ]:
schematic = schemdraw.Drawing() # Instantiates the drawing
ynew = 4
if (coord.get(v0[0])[1] < 0):
    ynew = -4
schematic += elm.SourceV().endpoints(coord.get(v0[0]), (coord.get(v0[0])[0], 
coord.get(v0[0])[1]+ynew)).label(str(v0[1])+'V') # Creates the source at the v0 node

for i in coord.keys():
    schematic += elm.Line().endpoints(coord.get(i), 
    coord.get(i)).label(i, color = "#48cae4", loc = "center") 
    # Creates a node at each coordinate
schematic += elm.Ground().at(coord.get(sink)) # Creates the sink node at the given node_id

for i in range(num_edges):
    index=[coord.get(node_ids[list(a[i]).index(-1)]),coord.get(node_ids[list(a[i]).index(1)])]
    schematic += (R1 := elm.Resistor().endpoints(index[0], index[1])) 
    # Resistance is found at the diagonal of the given indices
    schematic += elm.CurrentLabelInline(direction='in').at(R1).label('$'+
            str(int(r[i,i]))+'\Omega$', rotate = True , color = "#023e8a")
    # Resistance is found at the diagonal of the given 
schematic.draw()


# Concatenating the matrices
* The original linear system is
$$\begin{pmatrix}A & R\cr 0 & A^T\cr\end{pmatrix}
\begin{pmatrix}v\cr c\cr\end{pmatrix}=0\newline$$


* However,  $v_0$ and $v_n$ are already known, so the columns of the Node-Arc Incidence Matrix and transfer them to the zero vector
    * Using the source connection dictionary, the corresponding rows of the zero vector are updated to include the source voltage
* The source and sink currents do not sum to zero, so the corresponding rows are removed

In [ ]:
a1 = np.delete(a, [node_ids.index(v0[0]), node_ids.index(sink)], axis = 1)
arr1 = np.concatenate([a1,r], axis = 1, dtype = float) # Top row of the array is concatenated
arr2 = np.concatenate([np.zeros((num_nodes-2, num_nodes-2)), a1.T], axis = 1, dtype = float) 
# Bottom row of the array is concatenated
arr = np.concatenate([arr1, arr2], axis = 0, dtype = float) # Top and bottom rows concatenated
rhs = np.zeros(num_nodes + num_edges - 2) # Solution is 0s
start_node = [node_ids.index(v0[0])]
# Fix the rhs using the source_connect dictionary
for i in source_connect:
    rhs[i] = source_connect.get(i)

#rhs[:4] = rhs[:4] - (a[:, node_ids.index(v0[0])]) * v0[1]

print(arr)  
print(rhs)


In [ ]:
print("Source Connect Dictionary: "+str(source_connect))
print("Updated Zero Vector: "+str(rhs))

# Singular Value Decomposition of System

In [ ]:
# Singular Value Decomposition
u, s, vt = np.linalg.svd(arr)
utb = u.T @ rhs / s
soln = sum(utb[i] * vt[i] for i in range(len(utb)))

# Formatting the voltages
if (node_ids.index(v0[0]) < node_ids.index(sink)):
    voltages = soln[:node_ids.index(v0[0])] # Everything before the source node
    voltages = np.append(voltages, v0[1]) # Source node
    voltages = np.append(voltages, soln[node_ids.index(v0[0]):node_ids.index(sink)-1]) 
    # Between
    voltages = np.append(voltages, 0) # Sink node
    voltages = np.append(voltages, soln[node_ids.index(sink)-1:num_nodes-2]) 
    # Everything before the sink node
else:
    voltages = soln[:node_ids.index(sink)] # Everything before the source node
    voltages = np.append(voltages, 0) # Source node
    voltages = np.append(voltages, soln[node_ids.index(sink):node_ids.index(v0[0])-1]) 
    # Between
    voltages = np.append(voltages, v0[1]) # Sink node
    voltages = np.append(voltages, soln[node_ids.index(v0[0])-1:num_nodes-2]) 
    # Everything before the sink node

currents = soln[num_nodes-2:] # Currents

# Printing the x vector to check
print("Voltages")
for i in range(num_nodes):
    print(node_ids[i], str(voltages[i]), "volts")
print("\nCurrents")
for i in range(num_edges):
    print(node_ids[list(a[i]).index(-1)], "-->", node_ids[list(a[i]).index(1)], currents[i], "amps")
    

# Definition of the Schematic Drawing

In [ ]:
def draw_schem(vers, schematic):
    ynew = 4
    if (coord.get(v0[0])[1] < 0):
        ynew = -4
    schematic += elm.SourceV().endpoints(coord.get(v0[0]), (coord.get(v0[0])[0], 
    coord.get(v0[0])[1]+ynew)).label(str(v0[1])+'V') 
    # Creates the source at the v0 node
    for i in coord.keys(): # Node IDs
        if vers.lower() == "voltage":
            schematic += elm.Line().endpoints(coord.get(i), coord.get(i)).label(i+
            " - "+str(voltages[node_ids.index(i)])[:3]+"V", color = "#48cae4", loc = "center")
        else :
            schematic += elm.Line().endpoints(coord.get(i), coord.get(i)).label(i,
                                                color = "#48cae4", loc = "center") 
        # Creates a node at each coordinate
        schematic += elm.Ground().at(coord.get(sink)) 
        # Creates the sink node at the given node_id
    
    for i in range(num_edges): 
        index=[coord.get(node_ids[list(a[i]).index(-1)]),
                coord.get(node_ids[list(a[i]).index(1)])]
            # Index of edges found by which indices of the node-arc array are -1 and 1
        if vers.lower() == "current":
            schematic += (R1 := elm.Resistor().endpoints(index[0], index[1])).label('$'+
            str(int(r[i,i]))+'\Omega$', loc = "bottom", color = "#023e8a") 
            # Resistance is found at the diagonal of the given indices
            schematic += elm.CurrentLabelInline(direction='in').at(R1).label(
                str(currents[i])[:5], color = "#0096c7", rotate = True)
        else:
            schematic += (R1 := elm.Resistor().endpoints(index[0], index[1]))

    return schematic

## Voltage Drawing

In [ ]:
draw_schem("voltage", schemdraw.Drawing()).draw()

## Current Drawing

In [ ]:
draw_schem("current", schemdraw.Drawing()).draw()

# Verification:
The voltage drop along each edge is equal to the current times the resistance

$$Av=-Rc$$

In [ ]:
print("Predicted: " + str(a @ voltages)+"\n")
print("Expected: "+str(-r @ currents)+"\n")
print("Residuals: "+str(a @ voltages + r @ currents))

* The sum of the currents flowing into each node must be zero

$$A^Tc=0$$

In [ ]:
print("Residuals: " + str(a1.T @ currents) +"\n")